# Step 3: Pretrain - 预训练语言模型

## 学习目标

1. 理解预训练的目标：**下一个词预测**
2. **动手实现**数据集的构建
3. **动手实现**学习率调度（Warmup + Cosine Decay）
4. **动手实现**训练循环

## 学习方式

1. 在这个 Notebook 中学习概念
2. 去 `data_exercise.py` 完成数据集 TODO
3. 去 `train_exercise.py` 完成训练 TODO
4. 卡住了？查看 `*_solution.py`

---

## 1. 语言建模：下一个词预测

GPT 的预训练目标很简单：**给定前文，预测下一个词**

```
输入: "今天天气"
目标: "天气真好"

位置 0: 输入"今" -> 预测"天"
位置 1: 输入"今天" -> 预测"气"
位置 2: 输入"今天天" -> 预测"真"
位置 3: 输入"今天天气" -> 预测"好"
```

损失函数是**交叉熵**：
```python
loss = -sum(log P(下一个词 | 前文))
```

In [ ]:
import torch
import torch.nn.functional as F
import math

# 演示：语言建模目标
# 假设词表大小为 10，序列长度为 4

# 模型输出的 logits（每个位置预测下一个词的分数）
logits = torch.randn(1, 4, 10)  # [batch, seq_len, vocab_size]

# 目标序列（真实的下一个词）
targets = torch.tensor([[2, 5, 3, 8]])  # [batch, seq_len]

# 计算交叉熵损失
loss = F.cross_entropy(
    logits.view(-1, 10),  # [batch * seq_len, vocab_size]
    targets.view(-1)      # [batch * seq_len]
)

print(f"Logits shape: {logits.shape}")
print(f"Targets shape: {targets.shape}")
print(f"Loss: {loss.item():.4f}")
print(f"\n理论最小 loss（完美预测）: 0")
print(f"随机预测的 loss: {math.log(10):.4f}")

---

## 2. 数据集构建

### 2.1 输入和目标的关系

```
文本:  h e l l o   w o r l d
位置:  0 1 2 3 4 5 6 7 8 9 10

样本 0 (block_size=4):
  输入: [h, e, l, l]  (位置 0-3)
  目标: [e, l, l, o]  (位置 1-4)
  
样本 1:
  输入: [e, l, l, o]  (位置 1-4)
  目标: [l, l, o,  ]  (位置 2-5)
```

**关键**：目标是输入右移一位！

In [ ]:
# 演示数据集构建
text = "hello world"
block_size = 4

# 简单的字符编码
chars = sorted(set(text))
char_to_idx = {c: i for i, c in enumerate(chars)}
data = torch.tensor([char_to_idx[c] for c in text])

print(f"文本: {text}")
print(f"编码: {data.tolist()}")
print(f"词表: {char_to_idx}")
print()

# 构建样本
for idx in range(3):
    x = data[idx : idx + block_size]
    y = data[idx + 1 : idx + block_size + 1]
    
    x_text = ''.join([chars[i] for i in x.tolist()])
    y_text = ''.join([chars[i] for i in y.tolist()])
    
    print(f"样本 {idx}: 输入='{x_text}' -> 目标='{y_text}'")

### 2.2 练习：实现 __getitem__

去 `data_exercise.py`，完成 **TODO 1**：实现数据集的 `__getitem__` 方法

In [ ]:
# 测试你的实现
import importlib
import data_exercise
importlib.reload(data_exercise)

# 创建示例数据（如果不存在）
import os
if not os.path.exists("sample_data.txt"):
    data_exercise.create_sample_data("sample_data.txt")

# 测试数据集
data_exercise.test_dataset()

---

## 3. 学习率调度

### 3.1 为什么需要学习率调度？

**Warmup（预热）**：
- 训练初期，模型参数是随机的
- 梯度可能很大且不稳定
- 从小学习率开始，逐渐增大

**Cosine Decay（余弦衰减）**：
- 训练后期需要更小的学习率
- 进行精细调整，收敛到更好的最优点

```
lr
↑
|   /\
|  /  \
| /    \___________
|/
+------------------→ step
  warmup   decay
```

In [ ]:
import matplotlib.pyplot as plt

# 演示学习率调度
def demo_lr_schedule(warmup_steps, total_steps, max_lr, min_lr):
    steps = list(range(total_steps))
    lrs = []
    
    for step in steps:
        if step < warmup_steps:
            # Warmup: 线性增加
            lr = max_lr * (step + 1) / warmup_steps
        else:
            # Cosine decay
            progress = (step - warmup_steps) / (total_steps - warmup_steps)
            lr = min_lr + 0.5 * (max_lr - min_lr) * (1 + math.cos(math.pi * progress))
        lrs.append(lr)
    
    return steps, lrs

# 绘制
steps, lrs = demo_lr_schedule(100, 1000, 1e-3, 1e-4)

plt.figure(figsize=(10, 4))
plt.plot(steps, lrs)
plt.axvline(x=100, color='r', linestyle='--', label='Warmup 结束')
plt.xlabel('Step')
plt.ylabel('Learning Rate')
plt.title('Warmup + Cosine Decay')
plt.legend()
plt.grid(True)
plt.show()

### 3.2 练习：实现学习率调度

去 `train_exercise.py`，完成 **TODO 2a, 2b**：

- 2a: Warmup 阶段的线性增加
- 2b: Cosine Decay 阶段的衰减

In [ ]:
# 测试学习率调度
import train_exercise
importlib.reload(train_exercise)

train_exercise.test_lr_schedule()

---

## 4. 训练循环

### 4.1 核心步骤

```python
for batch in dataloader:
    # 1. 前向传播
    logits, loss = model(input_ids, targets=labels)
    
    # 2. 反向传播
    loss.backward()
    
    # 3. 梯度裁剪（防止梯度爆炸）
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    # 4. 更新参数
    optimizer.step()
    optimizer.zero_grad()
```

### 4.2 练习：实现训练循环

去 `train_exercise.py`，完成 **TODO 3a, 3b, 3c**：

- 3a: 更新学习率
- 3b: 前向传播
- 3c: 反向传播 + 梯度裁剪 + 优化器更新

In [ ]:
# 运行训练（完成 TODO 后）
# 注意：需要先完成 step2 的 model_solution.py
# !python train_exercise.py --device cpu --epochs 2 --batch_size 8

---

## 5. 梯度裁剪

### 为什么需要梯度裁剪？

- 深层网络中，梯度可能会"爆炸"（变得非常大）
- 大梯度会导致参数更新过大，破坏已学到的知识
- 梯度裁剪限制梯度的最大范数

In [ ]:
# 演示梯度裁剪
import torch.nn as nn

# 创建一个简单模型
model = nn.Linear(10, 10)

# 模拟一些梯度
x = torch.randn(1, 10)
y = model(x).sum()
y.backward()

# 手动放大梯度（模拟梯度爆炸）
for p in model.parameters():
    if p.grad is not None:
        p.grad *= 100

# 计算梯度范数
grad_norm_before = torch.nn.utils.clip_grad_norm_(model.parameters(), float('inf'))
print(f"裁剪前梯度范数: {grad_norm_before:.2f}")

# 重新计算并裁剪
model.zero_grad()
y = model(x).sum()
y.backward()
for p in model.parameters():
    if p.grad is not None:
        p.grad *= 100

grad_norm_after = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
print(f"裁剪后梯度范数: {grad_norm_after:.2f}")

---

## 6. 运行完整训练

完成所有 TODO 后，运行完整的预训练：

In [ ]:
# 运行完整训练
!python train_exercise.py --device cpu --epochs 3 --batch_size 8 --log_interval 20

---

## 7. 验证清单

完成本步骤后，你应该能够：

- [ ] 解释"下一个词预测"的训练目标
- [ ] 实现数据集的 `__getitem__`
- [ ] 解释为什么需要学习率调度（Warmup + Decay）
- [ ] 实现学习率调度函数
- [ ] 实现完整的训练循环

---

## 下一步

预训练完成后，进入 [Step 4: SFT](../step4_sft/) 学习如何让模型学会遵循指令。